In [2]:
import sys

In [3]:
import os
print(os.getcwd())

/Users/qiwei/Desktop/Desktop - Qiwei’s MacBook Pro M1/Job/RacketGo/tennis-analysis-test-1


In [4]:
# 获取项目根目录的路径
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))

# 将项目根目录添加到 sys.path
if project_root not in sys.path:
    sys.path.append(project_root)

In [5]:
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import cv2
from utils.video_utils import read_video, save_video
from video_cut import VideoCut
from trackers import PlayerTracker, BallTracker

In [13]:
input_video_path = "input/tennis-test.mp4"
video_frames, fps = read_video(input_video_path)

视频原始帧率: 60.0 FPS


: 

In [7]:
# Detect Players and Ball
print("2. 初始化球员跟踪器...")
player_tracker = PlayerTracker(model_path="models/yolov8x.pt")
print("3. 初始化球跟踪器...")
ball_tracker = BallTracker(model_path='models/yolo9_best.pt')

2. 初始化球员跟踪器...
3. 初始化球跟踪器...


In [8]:
print("4. 开始检测球员...")
player_detections = player_tracker.detect_frames(video_frames, 
                                                read_from_stub=False, 
                                                stub_path="tracker_stubs/player_detections.pkl")
print(f"球员检测完成，检测到{len(player_detections)}帧的球员")

print("5. 开始检测球...")
ball_detections = ball_tracker.detect_frames(video_frames, 
                                            read_from_stub=False, 
                                            stub_path="tracker_stubs/ball_detections.pkl")
print(f"球检测完成，检测到{len(ball_detections)}帧的球")

print("6. 开始插值球的位置...")
ball_detections = ball_tracker.interpolate_ball_positions(ball_detections)
print("球位置插值完成")

4. 开始检测球员...
球员检测完成，检测到678帧的球员
5. 开始检测球...
球检测完成，检测到678帧的球
6. 开始插值球的位置...
球位置插值完成


In [9]:
# Detect ball shots
print("11. 开始检测击球帧...")
ball_shot_frames = ball_tracker.get_ball_shot_frames(ball_detections)
print(f"击球帧检测完成: {ball_shot_frames}")

11. 开始检测击球帧...
击球帧检测完成: [84, 129, 170, 242, 283, 333, 367, 407, 427, 458, 470, 499, 549, 619]


/Users/qiwei/Desktop/Desktop - Qiwei’s MacBook Pro M1/Job/RacketGo/tennis-analysis-test-1/trackers/ball_tracker.py:81: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_ball_positions['ball_hit'].iloc[i] = 1
/Users/qiwei/Desktop/Desktop - Qiw

In [10]:
# Draw output
## Draw Player Bounding Boxes
output_video_frames= player_tracker.draw_bboxes(video_frames, player_detections)
output_video_frames= ball_tracker.draw_bboxes(output_video_frames, ball_detections)

## Draw frame number on top left corner
for i, frame in enumerate(output_video_frames):
    cv2.putText(frame, f"Frame: {i}",(10,30),cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)





In [11]:
# Save video
output_video_path = "output_videos/output_video.avi"
print("13. 开始保存视频...")
save_video(output_video_frames, output_video_path)

13. 开始保存视频...
使用原始视频帧率: 60.0 FPS
视频保存完成，总帧数: 678，帧率: 60.0 FPS


In [12]:


shot_cut_output = "output_videos/shotcut/shot1.avi"
video_cutter = VideoCut(output_video_path, shot_cut_output, ball_shot_frames)
video_cutter.cut_video()
print("14. 所有处理完成！")

Video Duration: 11.3s, FPS: 60, Total Frames: 678
14. 所有处理完成！
